In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from train_sft import PROMPT_DICT
import os
from typing import List

# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
def generate_input(instruction: str, input_str: str = "") -> str:
    prompt_input, prompt_no_input = (
        PROMPT_DICT["prompt_input"],
        PROMPT_DICT["prompt_no_input"],
    )

    if input_str != "":
        res = prompt_input.format_map({"instruction": instruction, "input": input})
    else:
        res = prompt_no_input.format_map({"instruction": instruction})

    return res

In [3]:
base_model_name_or_path = "internlm-7b"
lora_model_name_or_path = "best_rename_checkpoint-14967"#"output_refusev2/checkpoint-29934"  # /checkpoint-9695"


model = AutoModelForCausalLM.from_pretrained(
    base_model_name_or_path,
    torch_dtype="auto",
    # device_map="auto",
    # if model_args.model_name_or_path.find("falcon") != -1 else False
    trust_remote_code=True,
).cuda(0)

model = PeftModel.from_pretrained(model, model_id=lora_model_name_or_path)
model.eval()
print("ok")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

ok


In [4]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_name_or_path, trust_remote_code=True, padding_side="left"
)

In [5]:
# text_input = [
#     "你是谁",
#     # text1 = "写一篇介绍性文章，介绍您最喜欢的旅游目的地。"
#     "对给定的电影进行评级。\n电影名称：肖申克的救赎\n",
#     "给出两个选项，要求选择其中一个。 \n你更喜欢哪种颜色？红或蓝？\n",
#     "分析最近一年来全球气温趋势，并提供趋势预测。\n",
#     "根据给定的产品说明，写一篇500字左右的产品评测和建议。\n产品：Apple iPhone X\n",
#     "描述你最喜欢的一本书，并简要解释它为什么对你有影响。",
# ]

# text_input = [
#     "减肥只吃黄瓜可以嘛\n",
# ] * 10


def batch_generate_data(
    text_input: List[str], use_train_model: bool = True, temp: float = 0.7
):
    text_input_format = [generate_input(i) for i in text_input]
    batch_inputs = tokenizer.batch_encode_plus(
        text_input_format, padding="longest", return_tensors="pt"
    )
    batch_inputs["input_ids"] = batch_inputs["input_ids"].cuda()
    batch_inputs["attention_mask"] = batch_inputs["attention_mask"].cuda()

    if use_train_model:
        # with model.disable_adapter():
        outputs = model.generate(
            **batch_inputs,
            max_new_tokens=256,
            do_sample=True,
            temperature=temp,
            top_p=0.8,
        )
    else:
        with model.disable_adapter():
            outputs = model.generate(
                **batch_inputs,
                max_new_tokens=256,
                do_sample=True,
                temperature=temp,
                top_p=0.8,
            )
    outputs = tokenizer.batch_decode(
        outputs.cpu()[:, batch_inputs["input_ids"].shape[-1] :],
        skip_special_tokens=True,
    )

    return outputs


# outputvalue = batch_generate_data(text_input)
# outputvalue

### 使用lora微调的模型

In [12]:
text_input = [
    # "你是chatgpt吗\n"
    # "你和openai是什么关系，是openai训练出来的你嘛\n"
    "工作压力太大怎么办\n"
    # "解释人类尺度在绘画中的作用。\n"
    # "喝咖啡可以降低血糖吗\n"
    # "你是谁\n"
    # "你可以做什么\n"
    # "描述您的梦想工作，包括您的兴趣、技能和目标，并探讨如何实现它。\n"
    # "你有什么问题，我来回答。\n",
    # "描述周围环境的气味和颜色。\n"
    # "为一个新型电动汽车设计一个仪表盘界面，包括以下数据：速度、剩余电量、行驶里程、驾驶模式和导航。\n汽车品牌：Tesla\n"
# "完成一个回合制RPG游戏，让用户进行选择并根据选择展开游戏剧情，游戏时间为20分钟。\n"
# "为给定短文进行逆向转写。\n短文：你好，我是Belle。\n"
# "询问上海过去一周的天气情况。\n"
# "写一个简短的描述，介绍一下在Belle系统中如何生成文本。\n"
# "给出一篇文章，并根据用户提供的主题，自动将其编辑成用户想要的风格。\n一篇文章和想要的风格描述（例如“正式”、“幽默”、“简短”等等）。\n"
# "良睦路程序员和小埋的关系\n"
# "列出前10本你最喜欢的小说及作者。 \n"
# "根据给定的用户名和密码，获取用户信息。\n用户名：yuanzhoulvpi，密码：********\n"
# "播放一首来自Spotify的特定歌曲。\n歌曲名称和艺术家。\n"
# "请构建一个具有3个隐藏层和100个神经元的神经网络，使用ReLU激活函数和Adam优化器进行训练，并在MNIST数据集上进行测试，输出测试集的准确率和损失。\n"
# "厚礼蟹，作为良睦路程序员训练的机器人小埋\n"
# "真的不想活了\n"
] * 10
print(text_input[0])

工作压力太大怎么办



In [13]:
# lora 训练结果
batch_generate_data(text_input, use_train_model=True, temp=0.8)

['以下是缓解工作压力的建议：\n1. 给自己设定合理的工作目标和时间表，避免过度压力。\n2. 学会有效的时间管理，合理安排工作任务和休息时间。\n3. 寻求支持和帮助，与同事、家人或朋友交流，分享压力。\n4. 培养健康的生活方式，包括良好的睡眠、饮食和运动习惯。\n5. 寻找工作以外的兴趣爱好，放松身心，缓解压力。\n6. 寻求心理治疗或咨询，寻求专业帮助。',
 '1. 制定优先级清单，优先处理重要且紧急的任务；\n2. 学会说“不”，避免过度承诺和承担太多任务；\n3. 保持健康的生活方式，包括充足的睡眠、健康的饮食和适当的运动；\n4. 学会放松和缓解压力，例如通过冥想、瑜伽、阅读等方式；\n5. 寻求支持和帮助，与同事、家人或朋友交流，寻求他们的建议和帮助。',
 '1. 调整工作计划和时间管理，合理分配任务和休息时间；\n2. 寻找适当的减压方式，如运动、冥想、阅读等；\n3. 寻求同事或领导的帮助和支持，探讨解决问题的方案；\n4. 学习放松技巧，如深呼吸、渐进性肌肉松弛等；\n5. 建立良好的工作与生活平衡，合理分配时间，避免过度工作。',
 '1. 制定优先级清单，优先处理重要且紧急的任务。\n2. 学会委托任务，让团队成员分担部分工作。\n3. 寻找放松方式，如运动、冥想、阅读等。\n4. 与同事、家人、朋友沟通，寻求支持和建议。\n5. 学会时间管理，合理分配时间和精力。\n6. 接受自己无法完成所有任务的事实，并学会接受失败和挫折。\n7. 寻求专业帮助，如心理治疗、咨询等。',
 '1. 确定优先事项并制定计划，以确保能够合理分配时间和精力。\n2. 学会放松和缓解压力的技巧，如冥想、呼吸练习、运动等。\n3. 与同事和上级沟通，寻求支持和理解。\n4. 寻求专业帮助，如心理治疗或职业咨询。\n5. 维护健康的生活方式，如良好的睡眠、饮食和运动习惯。',
 '1. 制定计划：制定一个清晰的工作计划，优先处理最重要的任务，并确保按时完成任务。\n2. 调整时间：将工作时间分配到不同的任务上，并为自己留出休息时间。\n3. 寻求帮助：向同事、家人或朋友寻求帮助，以减轻工作压力。\n4. 放松自己：寻找适合自己的放松方式，如运动、阅读、听音乐等。\n5. 学习管理：学习如何有效地管理时间、任务和优先级，以减少工作压力。\n6. 寻求支持：寻求

In [14]:
# 原来的模型
batch_generate_data(text_input, use_train_model=False, temp=0.8)

[' \n- 工作压力太大，可能是工作任务量太大，或者领导安排任务不合理，导致工作压力过大。\n- 可以向领导提出自己的看法，或者申请调整工作任务量，或者申请换一份工作。\n- 也可以寻求同事的帮助，大家一起分担工作任务，或者互相帮助，互相支持。\n- 最后，可以寻求心理医生的帮助，通过心理咨询来缓解压力。\n',
 ' \n> 我最近工作压力太大了，感觉快要撑不住了。\n> 我现在需要找一个安静的地方放松一下，让自己能够平静下来。\n> 我有一个很好的朋友，他是一名心理咨询师，我可以向他寻求帮助。\n> 我打算去找他，向他倾诉自己的烦恼，并寻求他的建议。\n> 我相信，通过他的帮助，我可以更好地处理自己的压力，让自己能够更好地面对工作和生活。',
 ' \n工作压力太大怎么办\n1. 调整心态，放松心情\n2. 适当运动，舒缓压力\n3. 与家人朋友倾诉，缓解压力\n4. 适当休息，放松身心\n5. 寻找兴趣爱好，转移注意力',
 ' \n工作压力太大怎么办\n工作压力太大怎么办\n\n1. 工作压力太大怎么办\n2. 工作压力太大怎么办\n3. 工作压力太大怎么办\n4. 工作压力太大怎么办\n5. 工作压力太大怎么办\n6. 工作压力太大怎么办\n7. 工作压力太大怎么办\n8. 工作压力太大怎么办\n9. 工作压力太大怎么办\n10. 工作压力太大怎么办\n11. 工作压力太大怎么办\n12. 工作压力太大怎么办\n13. 工作压力太大怎么办\n14. 工作压力太大怎么办\n15. 工作压力太大怎么办\n16. 工作压力太大怎么办\n17. 工作压力太大怎么办\n18. 工作压力太大怎么办\n19. 工作压力太大怎么办\n20. 工作压力太大怎么办\n21. 工作压力太大怎么办\n22. 工作压力太大怎么办\n23. 工作压力太大怎么办\n24. 工作压力太大怎么办\n25. 工作压力太大怎么办\n26. 工作压力太大怎么办\n27. 工作压力太大怎么办\n28. 工作压力太大怎么办\n29. 工作压力太大怎么办\n30. 工作压力太大怎么办\n31. 工作压力',
 ' \n1. 如果工作压力太大，我们可以采取以下措施来缓解压力：\n- 与同事和朋友交谈，寻求支持和建议。\n- 锻炼身体，如散步、慢跑、游泳等。\n- 尝试一些放松的技巧，如深呼吸、冥想、瑜

In [16]:
model = model.merge_and_unload()
model.save_pretrained("internlm-7b-lml")

In [17]:
tokenizer.save_pretrained("internlm-7b-lml")

('internlm-7b-lml\\tokenizer_config.json',
 'internlm-7b-lml\\special_tokens_map.json',
 'internlm-7b-lml\\./tokenizer.model',
 'internlm-7b-lml\\added_tokens.json')

In [18]:
model.dtype

torch.float16